In [10]:

import os

import pandas as pd
from Bio import SeqIO

In [13]:
phatyp_data_dir = "E:\master\\final_project\data\my_data\convert2phatyp"
fasta_data_dir = "E:\master\\final_project\data\my_data\\fasta"

for k in range(2):
    if k == 0:
        data_type = "train"
    else:
        data_type = "test"

    for i in range(4):
        if i == 0:
            group = "100_400"
        elif i == 1:
            group = "400_800"
        elif i == 2:
            group = "800_1200"
        elif i == 3:
            group = "1200_1800"
        else:
            raise Exception

        for j in range(5):
            fold = j + 1

            phatyp_df = pd.read_csv(f"{phatyp_data_dir}/{group}/{fold}/{data_type}/ds.csv")

            sequence = []
            label = []
            id = []
            with open(f"{fasta_data_dir}/{group}/{fold}/{data_type}/data.fa", "r") as f:
                for record in SeqIO.parse(f, "fasta"):
                    sequence.append(str(record.seq))
                    id.append(record.id.split("_")[0] + record.id.split("_")[1])
                    if record.id.split("_")[2] == 'temperate':
                        label.append(0)
                    else:
                        label.append(1)
            fasta_df = pd.DataFrame({"sequence": sequence, "name": id, "label": label})
            # fasta_df.head()

            merge_df = pd.merge(phatyp_df, fasta_df, on="name")
            merge_df = merge_df[["name", "sequence", "label_y"]]
            columns = merge_df.columns.tolist()
            columns[2] = "target"
            merge_df.columns = columns
            # merge_df.head()

            output_dir = "E:\master\\final_project\data\my_data\\filtered_from_phatyp"
            data_dir = f"{output_dir}/{group}/{fold}/{data_type}"
            if not os.path.exists(data_dir):
                os.makedirs(data_dir)
            merge_df.to_csv(os.path.join(data_dir, f"data.csv"), index=False)